# Imports

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
def fix_layout(width:int=95):
    from IPython.core.display import display, HTML
    display(HTML('<style>.container { width:' + str(width) + '% !important; }</style>'))
    
fix_layout()

In [5]:
from nnpde.solve_area import Solve_Area
from tqdm.notebook import tqdm
import torch
import numpy as np
import matplotlib.pyplot as plt

In [6]:
from nnpde.utils.logs import enable_logging, logging 

# Setup

In [7]:
enable_logging(20)

#seed = 9 # Does not give problems
#torch.manual_seed(seed)
#np.random.seed(seed)

2021-07-08 13:03:40,530 - root - INFO - logs - logging enabled for level: 20


In [8]:
# log_prob has to be defined. also, backward of it will be calculated
my_dist = torch.distributions.multivariate_normal.MultivariateNormal(loc = torch.tensor([0., 0., 0.]), 
                                                                     covariance_matrix = torch.eye(3, dtype = torch.float32))

In [9]:
"""
examp = my_dist.sample(torch.Size([1000]))
plt.scatter(examp[:, 0], examp[:, 1])
plt.grid()
plt.show()
"""

'\nexamp = my_dist.sample(torch.Size([1000]))\nplt.scatter(examp[:, 0], examp[:, 1])\nplt.grid()\nplt.show()\n'

In [10]:
n_dims = 3 #specify dimensionality of data
cube_coords = torch.tensor([[-4., 4.], [-4., 4.], [-4., 4.]]) # specify coordinate borders of cube
N = torch.tensor([32, 32, 32]) # specify grid size of every dimension. 
# h (float) can be specified instead of N
# Note: distance between adjacent nodes h has to be the same across all dimensions

In [11]:
region_solver = Solve_Area(n_dims = n_dims, 
                           cube_coords = cube_coords, 
                           distribution = my_dist, 
                           N = N)

In [12]:
base_parameters = {
    "nb_layers": 3,
    "max_epochs": 300,
    "batch_size": 10,
    "stable_count": 10,
    "n_dims": n_dims,
    "optimizer": "Adadelta",
}

region_solver.train_model(base_parameters)

2021-07-08 13:04:41,376 - root - INFO - model - Using optimizer Adadelta


2021-07-08 13:04:42,142 - root - INFO - model - Training with max_epochs: 300, tol: 1e-06. Initial loss is 5361.84228515625
2021-07-08 13:04:45,596 - root - INFO - model - Epoch 0 with total loss 4576.7841796875
2021-07-08 13:10:22,304 - root - INFO - model - Epoch 100 with total loss 516.0802001953125
2021-07-08 13:15:59,577 - root - INFO - model - Epoch 200 with total loss 474.3822937011719
2021-07-08 13:21:31,765 - root - INFO - model - 299 epochs with total loss 458.0713195800781


## Test on distribution

In [22]:
def value_function(x):
    #x = [n_items, n_dims]
    #return (x**3).sum(dim=1)
    #return (torch.sin(x**3)).sum(dim=1)
    return torch.exp(1+x.sum(dim=1))

In [23]:
samps = my_dist.sample([10000])

In [ ]:
region_solver.solve_setting(function = value_function, samples = samps)

In [ ]:
def regular_expectancy(f, p, test_amnt, sample_amnt):
    results = list()
    for i in tqdm(range(test_amnt)):
        samples = p.sample([sample_amnt])
        mean = f(samples).mean()
        results.append(mean.item())
    return results

In [ ]:
t_am = 100
s_am = 10**3
reg_exp = regular_expectancy(value_function, my_dist, t_am, s_am)

In [ ]:
def spec_exp(f, test_amnt, sample_amnt, region_solver, p):
    results = list()
    for i in tqdm(range(test_amnt)):
        samples = p.sample([sample_amnt])
        cv = region_solver.get_cv(samples)
        mean = (f(samples) - cv).mean()
        results.append(mean.item())
    return results

In [ ]:
cv_exp = spec_exp(value_function, t_am, s_am, region_solver, my_dist)

In [ ]:
def box_comp(a, b):                                                    
    data = [a, b]                                             
    plt.figure(figsize=(12,8))                 
    plt.boxplot(data, showfliers = False, labels =                              
                ["MC Vanila", "Diff CV"])
    plt.grid()
    plt.show()

In [ ]:
box_comp(reg_exp, cv_exp)